# LSTM para lenguaje natural a nivel de caracteres 

### Objetivo: <br>
- Implementar una red neuronal de tipo LSTM (Long Short Term Memory) sin el uso de librerías especializadas. La única librería que uso es numpy.

![Alt text](img/LSTM_rnn.png "Title text")

- La imagen muestra la estructura de una red neuronal recurrente que utiliza unidades LSTM.
- Se puede observar que se repite la misma celda en diferentes pasos de tiempo.

![Alt text](img/LSTM.png "Title text")

* *t*: paso de tiempo.
* *c*: Memoria a largo plazo.
* *a*: Memoria a corto plazo. 
* Compuertas:
    1. Compuerta de olvido: Decide qué información de la memoria de largo plazo ($c^{\langle t-1 \rangle}$) es irrelevante y debe borrarse.<br>Toma cómo entradas la memoria anterior a corto plazo ($a^{\langle t-1 \rangle}$) y el dato actual ($x^{\langle t \rangle}$)
    2. Compuerta de Actualización: Decide que nueva información se va a guardar en la memoria de largo plazo.<br>
    Operaciones:
        * Se multiplican los vectores de la compuerta de actualización y de $\tilde{c}^{\langle t \rangle}$ y despúes sumamos el resultado a $c^{\langle t \rangle}$.
    3. Compuerta de Salida: Decide qué va a salir de la celda en este momento (el nuevo $a^{\langle t \rangle}$)
    Operaciones:
        * Toma la memoria de largo plazo ($c$) y la pasa por una tanh para normalizarla (-1 a 1).
        * Filtra esa memoria usando una sigmoide (compuerta de salida).
        * El resultado es el nuevo estado oculto $a^{\langle t \rangle}$.
* Predicción ($y^{\langle t \rangle}$): El estado oculto $a^{\langle t \rangle}$ entra a una capa densa con activación Softmax. En este caso es un vector de probabilidad de 52 dimensiones (caracteres válidos) que dice cuál es la letra más probable que sigue.

In [52]:
import numpy as np
import os

* Función de activación para todas las compuertas.
$\sigma(x) = \frac{1}{1+e^{-x}}$.

In [53]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

- Función que utilizamos para la predicción: $$ \text{softmax}(x) = \frac{e^{x - \max(x)}}{\sum e^{x - \max(x)}} $$ donde *x* es $a^{\langle t \rangle}$

In [54]:
def softmax(x):
    e_x = np.exp(x - np.max(x)) #Restamos el máximo para estabilidad numérica.
    return e_x / e_x.sum(axis=0)

## Forward Propagation (cómo hacemos predicciones)

- Trabajamos con muchos pesos y para que nuestra red neuronal aprenda es necesario inicializar de manera aleatoria los pesos. 
- Los guardamos en las siguientes matrices: (**n_a**: unidades (neuronas) ocultas (100), **n_x**: unidades de entrada (52), **n_y**: unidades de salida (52))
    * *$W_{f}$*: Matriz con los pesos de la compuerta de olvido. *Dim*: (n_a, n_a + n_x) = (100, 152).
    * *$b_{f}$*: Vector con los bias de la compuerta de olvido. *Dim*: (n_a, 1) = (100, 1).
    * *$W_{i}$*: Matriz con los pesos de la compuerta de actualización. *Dim*: (n_a, n_a + n_x) = (100, 152).
    * *$b_{i}$*: Vector con los bias de la compuerta de actualización. *Dim*: (n_a, 1) = (100, 1).
    * *$W_{c}$*: Matriz con los pesos de la compuerta candidata ($\tilde{c}^{\langle t \rangle}$). *Dim*: (n_a, n_a + n_x) = (100, 152).
    * *$b_{c}$*: Vector con los bias de la compuerta candidata ($\tilde{c}^{\langle t \rangle}$). *Dim*: (n_a, 1) = (100, 1)
    * *$W_{o}$*: Matriz con los pesos de la compuerta de salida. *Dim*: (n_a, n_a + n_x) = (100, 152).
    * *$b_{o}$*: Vector con los bias de la compuerta de salida. *Dim*: (n_a, 1) = (100, 1)
    * *$W_{y}$*: Matriz con los pesos para la predicción. *Dim*: (n_y, n_a) = (52, 100).
    * *$b_{y}$*: Vector con los bias para la predicción. *Dim*: (n_y, 1) = (52, 1).


In [55]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Inicializa los parámetros para la LSTM
    """
    np.random.seed(1)
    parameters = {}
    
    # Xavier initialization (escala los pesos para que no sean muy grandes)
    std = np.sqrt(2.0 / (n_a + n_x))
    
    # Pesos y sesgos para las puertas (Forget, Update, Output, Candidate)
    parameters['Wf'] = np.random.randn(n_a, n_a + n_x) * std
    parameters['bf'] = np.zeros((n_a, 1))
    parameters['Wi'] = np.random.randn(n_a, n_a + n_x) * std
    parameters['bi'] = np.zeros((n_a, 1))
    parameters['Wc'] = np.random.randn(n_a, n_a + n_x) * std
    parameters['bc'] = np.zeros((n_a, 1))
    parameters['Wo'] = np.random.randn(n_a, n_a + n_x) * std
    parameters['bo'] = np.zeros((n_a, 1))
    
    # Pesos para la predicción (Output layer)
    parameters['Wy'] = np.random.randn(n_y, n_a) * std
    parameters['by'] = np.zeros((n_y, 1))
    
    return parameters

* En cada celda de nuestra red neuronal suceden las siguientes operaciones.
1. Concatenamos $a^{\langle t-1 \rangle}$ con $x^{\langle t \rangle}$.
2. Calculamos las compuertas.
    - Compuerta de olvido: $$ \Gamma_f^{\langle t \rangle} = \sigma(W_f \cdot [a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f) $$
    - Compuerta de actualización: $$ \Gamma_u^{\langle t \rangle} = \sigma(W_i \cdot [a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_i) $$
    - Compuerta candidata: $$ \tilde{c}^{\langle t \rangle} = \tanh(W_c \cdot [a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c) $$
        - Actualizamos $c^{\langle t \rangle}$ usando $\tilde{c}^{\langle t \rangle}$, $\Gamma_u^{\langle t \rangle}$ y $\Gamma_f^{\langle t \rangle}$: <br>$$c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}\cdot c^{\langle t-1 \rangle}+\Gamma_u^{\langle t \rangle}\cdot \tilde{c}^{\langle t \rangle}$$
- Compuerta Oculta: $$\Gamma_o^{\langle t \rangle}=\sigma(W_o \cdot [a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)$$
- Activación $$a^{\langle t \rangle}=\Gamma_o^{\langle t \rangle}\cdot \tanh(c^{\langle t \rangle})$$
- **Predicción:** $$y^{\langle t \rangle}=\text{softmax}(W_y \cdot a^{\langle t \rangle}+b_y)$$

In [56]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Un solo paso de tiempo del LSTM.
    """
    # Recuperar parámetros
    Wf = parameters["Wf"]; bf = parameters["bf"]
    Wi = parameters["Wi"]; bi = parameters["bi"]
    Wc = parameters["Wc"]; bc = parameters["bc"]
    Wo = parameters["Wo"]; bo = parameters["bo"]
    Wy = parameters["Wy"]; by = parameters["by"]
    
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    # 1. Concatenar a_prev y xt
    concat = np.concatenate((a_prev, xt), axis=0)

    # 2. Calcular compuertas
    ft = sigmoid(np.dot(Wf, concat) + bf)        # Forget gate
    it = sigmoid(np.dot(Wi, concat) + bi)        # Update gate
    cct = np.tanh(np.dot(Wc, concat) + bc)       # Candidate value
    c_next = ft * c_prev + it * cct              # Cell state update
    ot = sigmoid(np.dot(Wo, concat) + bo)        # Output gate
    a_next = ot * np.tanh(c_next)                # Hidden state output
    

    yt_pred = softmax(np.dot(Wy, a_next) + by)

    # Guardar caché para backprop
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

* La celda anterior es para calcular un paso de tiempo dentro de la red neuronal, la siguiente función ejecuta el LSTM sobre todos los pasos de tiempo ($T_x$).
* Para nuestra arquitectura, utilizaremos $T_x=25$, lo que significa que la red neuronal mira una ventana de 25 caracteres pasados para intentar predecir el siguiente. 

In [57]:
def lstm_forward(x, a0, parameters):
    """
    Ejecuta el LSTM sobre todos los pasos de tiempo T_x.
    """
    caches = []
    n_x, m, T_x = x.shape #(52, 1, 25) = (número de caracteres, batch size, time steps).
    n_y, n_a = parameters['Wy'].shape #(52, 100)
    
    # Inicializar tensores de salida
    a = np.zeros((n_a, m, T_x)) #(100, 1, 25)
    c = np.zeros((n_a, m, T_x)) #(100, 1, 25)
    y = np.zeros((n_y, m, T_x)) #(52, 1, 25)
    
    a_next = a0 
    c_next = np.zeros((n_a, m)) #(100, 1)
    
    # Loop sobre el tiempo
    for t in range(T_x):
        xt = x[:,:,t]
        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, parameters)
        
        a[:,:,t] = a_next
        c[:,:,t] = c_next
        y[:,:,t] = yt
        caches.append(cache)
        
    caches = (caches, x)
    return a, y, c, caches

## Backpropagation (cómo ajustamos los pesos)

#### Para actualizar los pesos usamos el descenso del gradiente, esto nos sirve para minimizar la función de perdida.

* La función de perdida que utilizamos es la entropía cruzada categórica dado que es un problema de clasificación multiclase.

$$ L = - \log(\hat{y}_k) $$

In [58]:
def compute_loss(y_hat, y_indices):
    """
    Calcula la pérdida Cross-Entropy
    """
    loss = 0
    # y_hat tiene shape (n_y, m, T_x). Aquí m=1
    for t in range(len(y_indices)):
        # Probabilidad que la red asignó al carácter correcto
        prob = y_hat[y_indices[t], 0, t] 
        loss -= np.log(prob)
    return loss

#### Cálculo de los gradientes para un solo time step:
Lo que queremos encontrar es lo siguiente: $$\frac{\partial L}{\partial W_f}, \frac{\partial L}{\partial W_i}, \frac{\partial L}{\partial W_c}, \frac{\partial L}{\partial W_o}$$ $$\frac{\partial L}{\partial b_f}, \frac{\partial L}{\partial b_i}, \frac{\partial L}{\partial b_c}, \frac{\partial L}{\partial b_o}$$


#### Es necesario utilizar la regla de la cadena:
$$\frac{\partial L}{\partial W_f}=\frac{\partial L}{\partial c^{\langle t \rangle}} \frac{\partial c^{\langle t \rangle}}{\partial \Gamma_f^{\langle t \rangle}} \frac{\partial \Gamma_f^{\langle t \rangle}}{\partial z_f^{\langle t \rangle}} \frac{\partial z_f^{\langle t \rangle}}{\partial W_f}$$

$$\frac{\partial L}{\partial W_i} = \frac{\partial L}{\partial c^{\langle t \rangle}} \frac{\partial c^{\langle t \rangle}}{\partial \Gamma_i^{\langle t \rangle}} \frac{\partial \Gamma_i^{\langle t \rangle}}{\partial z_i^{\langle t \rangle}} \frac{\partial z_i^{\langle t \rangle}}{\partial W_i}$$

$$\frac{\partial L}{\partial W_c} = \frac{\partial L}{\partial c^{\langle t \rangle}} \frac{\partial c^{\langle t \rangle}}{\partial \tilde{c}^{\langle t \rangle}} \frac{\partial \tilde{c}^{\langle t \rangle}}{\partial z_c^{\langle t \rangle}} \frac{\partial z_c^{\langle t \rangle}}{\partial W_c}$$

$$\frac{\partial L}{\partial W_o} = \frac{\partial L}{\partial a^{\langle t \rangle}} \frac{\partial a^{\langle t \rangle}}{\partial \Gamma_o^{\langle t \rangle}} \frac{\partial \Gamma_o^{\langle t \rangle}}{\partial z_o^{\langle t \rangle}} \frac{\partial z_o^{\langle t \rangle}}{\partial W_o}$$

In [59]:
def lstm_cell_backward(da_next, dc_next, cache):
    """
    Calcula gradientes para un solo paso de tiempo.
    """
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
    n_x, m = xt.shape
    n_a, m = a_next.shape
    
    # Derivadas de las compuertas 
    dot = da_next * np.tanh(c_next) * ot * (1 - ot)
    dcct = (dc_next * it + ot * (1 - np.tanh(c_next)**2) * it * da_next) * (1 - cct**2)
    dit = (dc_next * cct + ot * (1 - np.tanh(c_next)**2) * cct * da_next) * it * (1 - it)
    dft = (dc_next * c_prev + ot * (1 - np.tanh(c_next)**2) * c_prev * da_next) * ft * (1 - ft)

    # Gradientes de los parámetros
    concat = np.concatenate((a_prev, xt), axis=0)
    dWf = np.dot(dft, concat.T)
    dWi = np.dot(dit, concat.T)
    dWc = np.dot(dcct, concat.T)
    dWo = np.dot(dot, concat.T)
    
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True)
    dbc = np.sum(dcct, axis=1, keepdims=True)
    dbo = np.sum(dot, axis=1, keepdims=True)

    # Gradientes para pasar al paso anterior (prev)
    # Recuperamos pesos Wf, Wi, etc.
    Wf, Wi, Wc, Wo = parameters['Wf'], parameters['Wi'], parameters['Wc'], parameters['Wo']
    
    da_prev = (np.dot(Wf[:, :n_a].T, dft) + np.dot(Wi[:, :n_a].T, dit) + 
               np.dot(Wc[:, :n_a].T, dcct) + np.dot(Wo[:, :n_a].T, dot))
               
    dc_prev = dc_next * ft + ot * (1 - np.tanh(c_next)**2) * ft * da_next
    
    dxt = (np.dot(Wf[:, n_a:].T, dft) + np.dot(Wi[:, n_a:].T, dit) + 
           np.dot(Wc[:, n_a:].T, dcct) + np.dot(Wo[:, n_a:].T, dot))
    
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, 
                 "dWf": dWf, "dbf": dbf, "dWi": dWi, "dbi": dbi,
                 "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}
    return gradients

* Dado que estamos trabajando con una red neuronal recurrente, es necesario calcular los gradientes a través del tiempo. Mientras que lstm_cell_backward calcula las derivadas de un solo instante, lstm_backward actúa como el coordinador temporal que recorre la secuencia desde el final hacia el principio.

In [60]:
def lstm_backward(da, caches):
    """
    Backprop a través del tiempo.
    """
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    # Inicializar gradientes acumulados
    dx = np.zeros((n_x, m, T_x))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    dc_prevt = np.zeros((n_a, m))
    
    dWf = np.zeros((n_a, n_a + n_x)); dWi = np.zeros_like(dWf)
    dWc = np.zeros_like(dWf); dWo = np.zeros_like(dWf)
    dbf = np.zeros((n_a, 1)); dbi = np.zeros_like(dbf)
    dbc = np.zeros_like(dbf); dbo = np.zeros_like(dbf)
    
    # Loop hacia atrás
    for t in reversed(range(T_x)):
        # Gradiente total en t = Gradiente externo (da) + Gradiente del futuro (da_prevt)
        gradients = lstm_cell_backward(da[:,:,t] + da_prevt, dc_prevt, caches[t])
        
        da_prevt = gradients["da_prev"]
        dc_prevt = gradients["dc_prev"]
        dx[:,:,t] = gradients["dxt"]
        
        # Acumular gradientes de pesos
        dWf += gradients["dWf"]; dWi += gradients["dWi"]
        dWc += gradients["dWc"]; dWo += gradients["dWo"]
        dbf += gradients["dbf"]; dbi += gradients["dbi"]
        dbc += gradients["dbc"]; dbo += gradients["dbo"]
        
    da0 = da_prevt

    gradients = {"dx": dx, "da0": da0, "dWf": dWf, "dbf": dbf, "dWi": dWi, "dbi": dbi,
                "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}
    return gradients


* Vamos a utilizar un optimizador para que el proceso del descenso del gradiente sea más eficiente

In [61]:
def initialize_adam(parameters):
    """
    Inicializa v y s como diccionarios con ceros.
    """
    L = len(parameters) // 2 # número de capas/tipos de parámetros
    v = {}
    s = {}
    
    for key in parameters.keys():
        v[key] = np.zeros(parameters[key].shape)
        s[key] = np.zeros(parameters[key].shape)
        
    return v, s

* Actualizamos los pesos (parámetros) utilizando Adam de la siguiente manera:

1. Cálculo del momento: media exponencial de los gradientes pasados:
$$v_t = \beta_1 v_{t-1} + (1 - \beta_1) \nabla J(\theta_t)$$
2. Corrección de sesgo de *v*:
$$\hat{v}_t = \frac{v_t}{1 - \beta_1^t}$$
3. Cálculo de s (RMSprop): media exponencial de los gradientes al cuadrado:
$$s_t = \beta_2 s_{t-1} + (1 - \beta_2) (\nabla J(\theta_t))^2$$
4. Corrección de sesgo para *s*:
$$\hat{s}_t = \frac{s_t}{1 - \beta_2^t}$$
5. **Actualización de parámetros:**
$$\theta_{t+1} = \theta_t - \alpha \frac{\hat{v}_t}{\sqrt{\hat{s}_t} + \epsilon}$$

In [62]:
def clip(gradients, maxValue):
    '''
    Recorta los gradientes para evitar explosión (Exploding Gradients)
    '''
    for gradient in [gradients[key] for key in gradients.keys()]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    return gradients

def update_parameters_with_adam(parameters, gradients, v, s, t, learning_rate=0.01, 
                                beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Actualiza parámetros usando Adam.
    """
    v_corrected = {}                         # Inicializar primera estimación de momento
    s_corrected = {}                         # Inicializar segunda estimación de momento
    
    for key in parameters.keys():
        # --- Cálculo de v (Momento) ---
        v[key] = beta1 * v[key] + (1 - beta1) * gradients['d' + key]
        # Corrección de sesgo para v
        v_corrected[key] = v[key] / (1 - np.power(beta1, t))
        
        # --- Cálculo de s (RMSprop) ---
        # Importante: gradients al cuadrado
        s[key] = beta2 * s[key] + (1 - beta2) * np.power(gradients['d' + key], 2)
        # Corrección de sesgo para s
        s_corrected[key] = s[key] / (1 - np.power(beta2, t))
        
        # --- Actualización de parámetros ---
        parameters[key] -= learning_rate * (v_corrected[key] / (np.sqrt(s_corrected[key]) + epsilon))

    return parameters, v, s

In [63]:
def save_checkpoint(parameters, v, s, file_name='checkpoint_kafka.npz'):
    """
    Guarda parámetros Y el estado del optimizador (v, s) en un solo archivo.
    Usa prefijos 'v_' y 's_' para distinguirlos.
    """
    save_dict = {}
    
    # 1. Guardar parámetros normales
    for key, value in parameters.items():
        save_dict[key] = value
        
    # 2. Guardar v (con prefijo v_)
    for key, value in v.items():
        save_dict['v_' + key] = value
        
    # 3. Guardar s (con prefijo s_)
    for key, value in s.items():
        save_dict['s_' + key] = value
        
    np.savez(file_name, **save_dict)
    print(f"¡Checkpoint guardado! (Params + Adam states) en: {file_name}")

def load_checkpoint(file_name='checkpoint_kafka.npz'):
    """
    Carga params, v y s separándolos por sus prefijos.
    """
    if not os.path.exists(file_name):
        print(f"No se encontró {file_name}. Se iniciará desde cero.")
        return None, None, None
    
    loaded = np.load(file_name)
    parameters = {}
    v = {}
    s = {}
    
    for key in loaded.files:
        if key.startswith('v_'):
            # Es parte de v (quitamos el prefijo 'v_')
            original_key = key[2:]
            v[original_key] = loaded[key]
        elif key.startswith('s_'):
            # Es parte de s (quitamos el prefijo 's_')
            original_key = key[2:]
            s[original_key] = loaded[key]
        else:
            # Es un parámetro normal
            parameters[key] = loaded[key]
            
    print("¡Checkpoint cargado correctamente! El optimizador recuerda dónde se quedó.")
    return parameters, v, s

In [64]:
def sample_top_k(parameters, char_to_ix, seed, temperature=1.0, k=5):
    """
    Genera texto usando Top-K Sampling para evitar bucles.
    k: Solo considera las k letras más probables en cada paso.
    """
    Wf, bf = parameters['Wf'], parameters['bf']
    Wi, bi = parameters['Wi'], parameters['bi']
    Wc, bc = parameters['Wc'], parameters['bc']
    Wo, bo = parameters['Wo'], parameters['bo']
    Wy, by = parameters['Wy'], parameters['by']
    
    vocab_size = by.shape[0]
    n_a = Wf.shape[0]
    
    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))
    c_prev = np.zeros((n_a, 1))
    
    # Inicializar
    if seed in char_to_ix:
        idx = char_to_ix[seed]
    else:
        idx = char_to_ix[' '] # Espacio por defecto si la semilla falla
        
    x[idx] = 1
    indices = []
    
    for _ in range(300): # Generar 300 caracteres
        # Forward
        a_next, c_next, y_hat, _ = lstm_cell_forward(x, a_prev, c_prev, parameters)
        
        # --- TOP-K LOGIC ---
        p = y_hat.ravel()
        
        # Aplicar temperatura
        p = np.log(p + 1e-10) / temperature
        p = np.exp(p) / np.sum(np.exp(p))
        
        # Ordenar probabilidades y quedarse con las top K
        top_k_indices = np.argsort(p)[-k:] # Índices de las k letras más probables
        top_k_probs = p[top_k_indices]
        
        # Renormalizar para que sumen 1
        top_k_probs = top_k_probs / np.sum(top_k_probs)
        
        # Elegir una de las top k
        choice = np.random.choice(top_k_indices, p=top_k_probs)
        
        indices.append(choice)
        
        # Preparar siguiente paso
        x = np.zeros((vocab_size, 1))
        x[choice] = 1
        a_prev = a_next
        c_prev = c_next
        
    return indices



## Definición del modelo

1. El modelo revisa si retoma los pesos que ya se habían entrenado o si empieza de cero. <br>
2. Entramos al bucle de entrenamiento: <br>
    * Preparamos los datos y utilizamos una ventana deslizante para moverse por el data set. <br>La entrada serán 25 caracters y el objetivo son también 25 caracteres pero desplazados un lugar a la derecha ya que queremos que nuestra red prediga el siguiente caracter dado el actual.<br>
    * One-hot encoding: convierte los caracteres a vectores de ceros y unos para que puedan ser procesados por la red neuronal.
3. **Forward propagation:** inicializamos en ceros el estado oculto inicial, pasamos los vectores one-hot por la red y obtenemos la predicción.
4. Calculamos la perdida utilizando un prmoedio móvil exponencial para suavizar (esto sirve sólo para la visulización).
5. **Backward propagation:** calculamos los gradientes para ver como deben de cambiar los pesos. 
6. Actualización de parámetros: modifica los pesos de la red utilizando Adam.
7. Cada 500 iteraciones imprimimos la pérdida, generamos texto y guardamos los pesos. 

In [65]:
def model(data, char_to_ix, ix_to_char, num_iterations=3000, n_a=50, vocab_size=52, 
          learning_rate=0.005, checkpoint_file='checkpoint_kafka.npz'):
    
    n_x, n_y = vocab_size, vocab_size #tamáño de los vocabularios (52 caracteres)
    
    #Cargamos los parámetros que ya teníamos de los entrenamientos pasados
    parameters, v, s = load_checkpoint(checkpoint_file) 
    
    # Si es la primera vez entrenando
    if parameters is None:
        print("Iniciando entrenamiento desde cero...")
        parameters = initialize_parameters(n_a, n_x, n_y) #Inicializamos parámetros aleatoriamente. 
        v, s = initialize_adam(parameters) #Inicializamos v y s cómo vectores con puros ceros. 
    else:
        print("Reanudando entrenamiento con estado previo...")
        # Si el archivo existía pero por alguna razón v o s no (versiones viejas), los reinicializamos por seguridad.
        if not v or not s:
            print("Advertencia: Se encontraron pesos pero no estado del optimizador. Reiniciando Adam.")
            v, s = initialize_adam(parameters)

    # Inicializar pérdida
    loss = -np.log(1.0/vocab_size)*25 #probabilidad totalmente al azar dado que todos los caracteres 
    
    # Bucle de entrenamiento
    for j in range(num_iterations):
        index = j % len(data)
        if index + 25 + 1 > len(data): index = 0
        single_example = data[index : index + 25]
        single_target = data[index + 1 : index + 26]
        X_indices = [char_to_ix[ch] for ch in single_example] #pasamos de caracter a entero
        Y_indices = [char_to_ix[ch] for ch in single_target] #pasamos de caracter a entero
        
        x_one_hot = np.zeros((n_x, 1, 25))
        for step, idx in enumerate(X_indices):
            x_one_hot[idx, 0, step] = 1

        # 1. Forward
        a0 = np.zeros((n_a, 1)) #estado oculto inicial en ceros
        a, y_hat, c, caches = lstm_forward(x_one_hot, a0, parameters) #pasa los datos por la red
        
        # 2. Loss 
        curr_loss = compute_loss(y_hat, Y_indices)
        if j == 0: 
            loss = curr_loss
        else: 
            loss = loss * 0.999 + curr_loss * 0.001
        
        # 3. Backward
        dy = np.copy(y_hat)
        for step, idx in enumerate(Y_indices):
            dy[idx, 0, step] -= 1
        gradients = lstm_backward(np.zeros_like(a), caches)
        
        # Gradientes capa salida
        gradients['dWy'] = np.zeros_like(parameters['Wy'])
        gradients['dby'] = np.zeros_like(parameters['by'])
        da = np.zeros_like(a)
        for step in reversed(range(len(X_indices))):
            gradients['dWy'] += np.dot(dy[:,:,step], a[:,:,step].T)
            gradients['dby'] += dy[:,:,step]
            da[:,:,step] = np.dot(parameters['Wy'].T, dy[:,:,step])
        grads_lstm = lstm_backward(da, caches)
        gradients.update(grads_lstm)
        
        # 4. Update
        gradients = clip(gradients, 5)
        # Usamos t = j + 1, pero idealmente guardaríamos 't' también. 
        # Para simplificar, reiniciar t no es tan grave como reiniciar v/s.
        parameters, v, s = update_parameters_with_adam(parameters, gradients, v, s, j+1, learning_rate)

        # Log y Guardado
        if j % 500 == 0:
            print(f"Iteración {j}, Pérdida: {loss:.4f}")
            try:
                seed = data[index]
                indices_generated = sample_top_k(parameters, char_to_ix, seed)
                print("Generado:", ''.join([ix_to_char[ix] for ix in indices_generated]).replace('\n', ' '))
            except:
                pass
            print('-'*50)
            
            # GUARDAR AQUÍ AUTOMÁTICAMENTE
            save_checkpoint(parameters, v, s, checkpoint_file)
            
    return parameters

In [66]:
def save_parameters(parameters, file_name='checkpoint_kafka.npz'):
    """
    Guarda el diccionario de parámetros en un archivo .npz
    """
    # El operador ** desempaqueta el diccionario para guardarlo con sus nombres clave
    np.savez(file_name, **parameters)
    print(f"¡Éxito! Parámetros guardados en: {file_name}")
    

def load_parameters(file_name='checkpoint_kafka.npz'):
    """
    Carga los pesos desde el archivo y reconstruye el diccionario.
    """
    if not os.path.exists(file_name):
        print(f"Error: No se encuentra el archivo {file_name}")
        return None
    
    # Cargar el archivo
    loaded = np.load(file_name)
    
    # Reconstruir el diccionario de Python (np.load devuelve un objeto especial)
    parameters = {}
    for key in loaded.files:
        parameters[key] = loaded[key]
        
    print("¡Pesos cargados correctamente! Tu red ya sabe escribir.")
    return parameters

In [67]:
# 1. Cargar diccionarios (del paso anterior que hicimos)
with open("metamorfosis_limpio.txt", 'r', encoding='utf-8') as f:
    texto = f.read()
chars = sorted(list(set(texto)))
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
# 2. Entrenar
parameters = model(texto, char_to_ix, ix_to_char, num_iterations=10000, n_a=100, vocab_size=vocab_size)
save_parameters(parameters)

¡Checkpoint cargado correctamente! El optimizador recuerda dónde se quedó.
Reanudando entrenamiento con estado previo...
Advertencia: Se encontraron pesos pero no estado del optimizador. Reiniciando Adam.
Iteración 0, Pérdida: 68.0088
Generado: asazrar de su enter ara nista y preferidal con caer en su esera quer prefería quera de ningún puese fueres en dera de ningún y genían ese podía doba podí el de no se dejada se heciento en esta perer colgando de esta ser en esta prefería quedentando de núera podía doberidar, perder logra modo es, pre
--------------------------------------------------
¡Checkpoint guardado! (Params + Adam states) en: checkpoint_kafka.npz
Iteración 500, Pérdida: 55.6805
Generado: o, precuas con el relantes perertido en coso de se desaba muchada. pensorde en en en punto sober, partión el sere teníación desalpulamente emen en para partes al sabe, patamententruido, con el cobírio ciónseño insoba, al teníarace de la cabeza sue sulgado se desbeve en compata, perorante co

In [68]:
# Cargar tus pesos entrenados
mis_pesos = load_parameters('checkpoint_kafka.npz')

# --- PRUEBA EL NUEVO SAMPLING ---
print("--- Generación Top-K (Más variada) ---")
idx = sample_top_k(mis_pesos, char_to_ix, seed='w', temperature=1, k=3)
print(''.join([ix_to_char[i] for i in idx]).replace('\n', ' '))

¡Pesos cargados correctamente! Tu red ya sabe escribir.
--- Generación Top-K (Más variada) ---
rese pues, en el angro esta posicabeza noger la cabeza, no peser le prinar en esta, el cuerpo de herida, era no perdento. perdentión lentar en esta, perdente pero recalmente, con lentrted de herida, pero la cabeza, no podía de prido dejando de prindo an de su erpeser en la cabeza, sación, se fuerza 
